In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset 
from torch.utils.data.dataloader import DataLoader
from PIL import Image
import matplotlib.pyplot as plt

img = Image.open('images\\megumin_explosion.jpg').convert('L')
pixels = img.getdata()

width, height = img.size

x_coords = []
y_coords = []
intensity = []
for i in range(height):
    for j in range(width):
        x_coords.append(j)
        y_coords.append(height-i)
        intensity.append(pixels[i*width + j])

df = pd.DataFrame({'x': np.array(x_coords)/width, 'y': np.array(y_coords)/height, 'z': np.array(intensity)/255.})

x = np.array(x_coords).reshape((height, width))
y = np.array(y_coords).reshape((height, width))
z = np.array(intensity).reshape((height, width))

plt.figure(figsize=(12,8))
plt.imshow(img, cmap='Greys_r')
plt.show()
# plt.imshow(z, cmap='Greys_r')
# plt.show()

x_tensor = torch.tensor(df.iloc[:,0:2].values, dtype=torch.float)
y_tensor = torch.tensor(df.iloc[:,-1].values, dtype=torch.float)

# Create an instance of the dataset and dataloader
batch_size = 1024 #1024*16
dataset = TensorDataset(x_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
print('Batches per Epoch:', df.index[-1]/batch_size)

print(x_tensor.shape)
print(y_tensor.shape)



ModuleNotFoundError: No module named 'numpy'

In [ ]:
# Define the neural network architecture
class Net(nn.Module):
    def __init__(self, nn_shape=(2, 4, 100, 1)):
        super(Net, self).__init__()
        self.num_inputs = nn_shape[0]
        self.num_layers = nn_shape[1]
        self.num_neurons = nn_shape[2]
        self.num_outputs = nn_shape[3]
        self.layers = nn.ModuleList() # create an empty list to store layers
        self.layers.append(nn.Linear(self.num_inputs, self.num_neurons)) # add the first layer
        
        for i in range(self.num_layers - 2): # add the remaining layers
            self.layers.append(nn.Linear(self.num_neurons, self.num_neurons))
        
        self.layers.append(nn.Linear(self.num_neurons, self.num_outputs)) # add the output layer
    
    def forward(self, x):
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        x = self.layers[-1](x)
        x = torch.sigmoid(x)
        return x
    
class AttentionNet(nn.Module):
    def __init__(self, nn_shape=(10,3,50,10), cond_inputs = 2, overlap=100):
        super(AttentionNet, self).__init__()
        #nn_shape = (n_inputs, n_layers, n_neurons, n_outputs)
        self.cond_inputs = cond_inputs
        self.num_inputs = nn_shape[0]
        self.num_layers = nn_shape[1]
        self.num_neurons = nn_shape[2]
        self.num_outputs = nn_shape[3]
        self.overlap = overlap
        self.qnet = Net(nn_shape=(self.cond_inputs,self.num_layers, self.num_neurons, self.num_inputs*self.overlap))
        self.knet = Net(nn_shape=(self.cond_inputs,self.num_layers, self.num_neurons, self.overlap*self.num_outputs))
        self.bnet = Net(nn_shape=(self.cond_inputs,self.num_layers, self.num_neurons, self.num_outputs))

    def calc_params(self, x):
        q = self.qnet(x)
        k = self.knet(x)
        q = torch.reshape(q, shape=(-1, self.num_inputs, self.overlap))
        k = torch.reshape(k, shape=(-1, self.overlap, self.num_outputs))
        w = torch.matmul(q,k)
        b = self.bnet(x)

        # print('q', q.size())
        # print('k', k.size())
        # print('w', w.size())
        # print('b', b.size())
        return w, b
    
    def forward(self, x, y):
        '''
        Passes in the layer input, x, and the data conditioning input, y, and
        then calculates layer weights and biases and uses them to compute layer outputs.
        '''
        # print('x', x.size())
        w, b = self.calc_params(y)
        # x = torch.matmul(w,x) + b
        x = torch.unsqueeze(x,dim=1)
        g = torch.matmul(x,w)
        g = torch.squeeze(g, dim=1)
        x = g + b

        return x


class NeuralNetNet(nn.Module):
    def __init__(self, nn_shape=(2,4,100,1), sub_net_shape=(3,100), overlap = 50):
        super(NeuralNetNet, self).__init__()
        self.num_inputs = nn_shape[0]
        self.num_layers = nn_shape[1]
        self.num_neurons = nn_shape[2]
        self.num_outputs = nn_shape[3]
        self.subnet_layers = sub_net_shape[0]
        self.subnet_neurons = sub_net_shape[1]
        self.overlap = overlap #number of "heads", this is the dimension eaten by the matrix mutliplication of keys and values

        self.sub_nets = nn.ModuleList() # create an empty list to store subnets
        self.sub_nets.append(AttentionNet(nn_shape=(self.num_inputs, self.subnet_layers, self.subnet_neurons, self.num_neurons),
                                          cond_inputs=self.num_inputs, overlap=self.overlap))
        for i in range(self.num_layers - 2): # add the remaining layers
            self.sub_nets.append(AttentionNet(nn_shape=(self.num_neurons, self.subnet_layers, self.subnet_neurons, self.num_neurons),
                                              cond_inputs=self.num_inputs, overlap=self.overlap))
        self.sub_nets.append(AttentionNet(nn_shape=(self.num_neurons, self.subnet_layers, self.subnet_neurons, self.num_outputs),
                                          cond_inputs=self.num_inputs, overlap=self.overlap))
        
    def forward(self, x):
        y = x
        for sub_net in self.sub_nets[:-1]:
            x = F.relu(sub_net(x, y))
        x = self.sub_nets[-1](x, y)
        x = torch.sigmoid(x)
        return x


# Create an instance of the neural network
# net = Net(nn_shape=(2,6,400,1))
# net = Net(nn_shape=(2,3,100,1))
net = NeuralNetNet(nn_shape=(2,4,50,1), sub_net_shape=(3,50), overlap = 50)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

def make_reduced_dataloader(scale=20):
    xlin = np.linspace(0, 1, width // scale)
    ylin = np.linspace(0, 1, height // scale)
    xv, yv = np.meshgrid(xlin, ylin)

    xv_tensor = torch.tensor(xv, dtype=torch.float).flatten()
    yv_tensor = torch.tensor(yv, dtype=torch.float).flatten()

    reduced_tensor = torch.stack((xv_tensor, yv_tensor), dim=-1)
    reduced_dataset = TensorDataset(reduced_tensor, torch.zeros(size=reduced_tensor.size()))
    reduced_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    return reduced_dataloader

# reduced_dataloader = make_reduced_dataloader(scale=4)

# Plot Raw Network Initialization
print('Plotting Raw Network Initialization')
def plot_performance(net, scale = 4):
    reduced_dataloader = make_reduced_dataloader(scale=scale)
    with torch.no_grad():
        output_list = []
        for x_batch, y_batch in reduced_dataloader:
            outputs = net(x_batch)
            output_list.append(outputs)
        outputs = torch.cat(output_list, dim=0)

    outputs = outputs.numpy()
    output_grid = outputs.reshape((height//scale, width//scale))
    plt.figure(figsize=(12,8))
    plt.imshow(output_grid, cmap='Greys_r', origin='lower')
    plt.show()

plot_performance(net)

# Train the network for 1000 epochs
loss_values = []
n_epochs = 4
print("Beginning Training")
for epoch in range(n_epochs):
    for g in optimizer.param_groups:
        # g['lr'] = (0.001/n_epochs)*(n_epochs - epoch) + 0.00001
        g['lr'] = 10**(-3. - 2.*(epoch/(n_epochs-1)))
    print('Epoch', epoch, 'of', n_epochs)
    for x_batch, y_batch in dataloader:
        print('x_batch',x_batch.size())
        # Forward pass
        outputs = net(x_batch)
        loss = criterion(outputs, y_batch.unsqueeze(dim=-1))
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Store the loss value
        loss_values.append(loss.item())

    # Print the loss
    print(f"Loss = {loss_values[-1]}")
    # Show current network progress
    if epoch in [0,1,2,4,10,20,40]:
        plot_performance(net)

# Save the trained model
torch.save(net.state_dict(), 'model2.pt')



# Plot the loss values
epochs_values = np.array(range(len(loss_values))) / np.ceil(df.index[-1]/batch_size)
plt.plot(epochs_values, loss_values)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.yscale('log')
plt.grid()
plt.show()


: 

In [ ]:
print(width, height)
xlin = np.linspace(0, 1, width // 2)
ylin = np.linspace(0, 1, height // 2)
xv, yv = np.meshgrid(xlin, ylin)

xv_tensor = torch.tensor(xv, dtype=torch.float).flatten()
yv_tensor = torch.tensor(yv, dtype=torch.float).flatten()

reduced_tensor = torch.stack((xv_tensor, yv_tensor), dim=-1)

# Print the loss
print(f"Loss = {loss_values[-1]}")

# with torch.no_grad():
#     outputs = net(x_tensor)
# outputs = outputs.numpy()
# output_grid = outputs.reshape((height, width))
# plt.figure(figsize=(12,8))
# plt.imshow(output_grid, cmap='Greys_r')

with torch.no_grad():
    outputs = net(reduced_tensor)
outputs = outputs.numpy()
output_grid = outputs.reshape((height//2, width//2))
plt.figure(figsize=(12,8))
plt.imshow(output_grid, cmap='Greys_r', origin='lower')

: 

In [ ]:
print(net)

: 

In [ ]:

print(np.log10(0.01))
print(np.log10(0.00001))
10**-2

: 

In [ ]:
x = torch.zeros(size=(16,2))
w = torch.zeros(size=(16,2,50))
b = torch.zeros(size=(16,50))

x = torch.unsqueeze(x,dim=1)
g = torch.matmul(x,w)
g = torch.squeeze(g, dim=1)
print('g',g.size())

: 